In [1]:
from flow_dataframe import flow_from_dataframe

import numpy as np
import pandas as pd
import os
import sys
from itertools import cycle

from keras.applications.nasnet import preprocess_input
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


Using TensorFlow backend.


In [2]:
model = load_model('bone_age_weights.best.hdf5')

# with tf.device('/cpu:0'):
#     model = load_model('results/mae_11/bone_age_weights.best.hdf5')

In [3]:
def combined_generators(image_generator, gender_data, batch_size):
    gender_generator = cycle(batch(gender_data, batch_size))
    while True:
        nextImage = next(image_generator)
        nextGender = next(gender_generator)
        assert len(nextImage[0]) == len(nextGender)
        yield [nextImage[0], nextGender], nextImage[1]

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [5]:
print('==================================================')
print('================ Evaluating Model ================')
print('==================================================')

BATCH_SIZE = 1
IMG_SIZE = (224, 224)
# IMG_SIZE = (299, 299)
gender_str_col = 'male'

base_dir = "rsna-bone-age"
test_df = pd.read_csv(os.path.join(base_dir, 'boneage-test-dataset.csv'))
test_df['path'] = test_df['fileName'].map(lambda x: os.path.join(base_dir,
                                                                 'boneage-test-dataset', 
                                                                 'boneage-test-dataset', 
                                                                 '{}'.format(x)))
# test_datagen = ImageDataGenerator(samplewise_center = True, 
#                                    samplewise_std_normalization = True, 
#                                    horizontal_flip = False, 
#                                    vertical_flip = False, 
#                                    height_shift_range = 0.1, 
#                                    width_shift_range = 0.1, 
#                                    rotation_range = 10, 
#                                    shear_range = 0.05,
#                                    fill_mode = 'nearest',
#                                    zoom_range = 0.15)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_df['exists'] = test_df['path'].map(os.path.exists)
test_df['gender'] = test_df['male'].map(lambda x: 'male' if x else 'female')

test_gen = flow_from_dataframe(test_datagen, test_df, 
                               path_col = 'path',
#                                y_col = 'bone_age_zscore', 
#                                class_mode = None,
                               target_size = IMG_SIZE,
                               color_mode = 'rgb',
                               batch_size = BATCH_SIZE)

gen_wrapper = combined_generators(test_gen, test_df[gender_str_col], BATCH_SIZE)

# pred = bone_age_div * model.predict_generator(test_gen, len(test_df), verbose = True) + bone_age_mean
pred = model.predict_generator(test_gen, len(test_df), verbose = True)
predictions = np.array(pred)
predictions = predictions.flatten()

print(predictions)

================ Evaluating Model ================
flow_from_dataframe() -->
## Ignore next message from keras, values are replaced anyways
Found 1262 images belonging to 1 classes.
Reinserting dataframe: 1262 images
flow_from_dataframe() <--
1262/1262 [==============================] - 72s 57ms/step
[22.178646 18.01988  18.433083 ... 20.947803 25.10832  17.593443]


In [6]:
filenames = test_df.fileName
results = pd.DataFrame({"fileName":filenames, "boneage":predictions})
results.to_csv("results.csv",index=False)

In [9]:
for i in predictions:
    print(i)

22.178646
18.01988
18.433083
19.286537
18.809267
21.188799
20.223305
25.260693
19.71023
17.980087
17.724073
18.670597
20.950394
18.35024
26.342442
22.111347
16.152311
18.154001
18.268951
20.650076
20.728615
25.333992
21.085695
20.05431
23.728405
21.574286
21.298826
18.26248
19.588104
21.318913
20.887001
17.80609
16.396544
21.405949
18.023129
22.673803
19.86642
20.675941
17.745062
15.779276
21.459621
19.001709
22.554926
22.680279
20.396666
14.503665
20.82905
21.162024
18.62243
20.349245
20.667603
18.276306
22.59496
17.467583
22.45019
35.5447
22.543728
20.49322
19.586658
18.391233
22.298597
18.174028
18.753983
18.575819
18.669584
21.541283
21.725685
23.384619
20.234447
16.550589
16.2587
19.309328
22.114037
17.874681
21.869864
20.608744
26.125578
22.532997
19.981096
18.06459
18.3399
15.557701
18.343483
17.951752
19.76499
18.098473
18.652712
22.28982
19.775099
20.04015
19.336641
20.815786
19.725782
19.391365
14.64593
16.419336
19.13473
21.377737
22.24818
18.610424
22.16252
21.314425
21.508